In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!wget -q http://apache.mirrors.pair.com/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
!ls

sample_data  spark-2.4.4-bin-hadoop2.7.tgz


In [0]:
!tar -xvf spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
!ls 

sample_data  spark-2.4.4-bin-hadoop2.7	spark-2.4.4-bin-hadoop2.7.tgz


In [0]:
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession, SQLContext
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
df = spark.createDataFrame([{"Google": "Colab","Spark": "Scala"} ,{"Google": "Dataproc","Spark":"Python"}])
df.show()

/content/spark-2.4.4-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+--------+------+
|  Google| Spark|
+--------+------+
|   Colab| Scala|
|Dataproc|Python|
+--------+------+



In [0]:
import string # to remove punctuation
import re
import pandas as pd
import os
import itertools
from collections import Counter
from pyspark.sql.functions import *
from pyspark.sql import functions as f
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import udf, col
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator


In [0]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

# Step 1: Load data

In [0]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls

drive  sample_data  spark-2.4.4-bin-hadoop2.7  spark-2.4.4-bin-hadoop2.7.tgz


In [0]:
#storing the file as dataframes
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
dataset = sqlContext.read.format("com.databricks.spark.csv").option("delimiter", "\t").load("/content/drive/My Drive/ml-100k/u.data")
type(dataset)

pyspark.sql.dataframe.DataFrame

In [0]:
dataset.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



In [0]:
dataset.show()

+---+----+---+---------+
|_c0| _c1|_c2|      _c3|
+---+----+---+---------+
|196| 242|  3|881250949|
|186| 302|  3|891717742|
| 22| 377|  1|878887116|
|244|  51|  2|880606923|
|166| 346|  1|886397596|
|298| 474|  4|884182806|
|115| 265|  2|881171488|
|253| 465|  5|891628467|
|305| 451|  3|886324817|
|  6|  86|  3|883603013|
| 62| 257|  2|879372434|
|286|1014|  5|879781125|
|200| 222|  5|876042340|
|210|  40|  3|891035994|
|224|  29|  3|888104457|
|303| 785|  3|879485318|
|122| 387|  5|879270459|
|194| 274|  2|879539794|
|291|1042|  4|874834944|
|234|1184|  2|892079237|
+---+----+---+---------+
only showing top 20 rows



In [0]:
#Changing column names
data = dataset.select(col("_c0").alias("user_id"), col("_c1").alias("item_id"), col("_c2").alias("rating"), col("_c3").alias("timestamp"))
data.show()

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
|    298|    474|     4|884182806|
|    115|    265|     2|881171488|
|    253|    465|     5|891628467|
|    305|    451|     3|886324817|
|      6|     86|     3|883603013|
|     62|    257|     2|879372434|
|    286|   1014|     5|879781125|
|    200|    222|     5|876042340|
|    210|     40|     3|891035994|
|    224|     29|     3|888104457|
|    303|    785|     3|879485318|
|    122|    387|     5|879270459|
|    194|    274|     2|879539794|
|    291|   1042|     4|874834944|
|    234|   1184|     2|892079237|
+-------+-------+------+---------+
only showing top 20 rows



In [0]:
df = data.select('user_id', 'item_id', 'rating')
df.show()

+-------+-------+------+
|user_id|item_id|rating|
+-------+-------+------+
|    196|    242|     3|
|    186|    302|     3|
|     22|    377|     1|
|    244|     51|     2|
|    166|    346|     1|
|    298|    474|     4|
|    115|    265|     2|
|    253|    465|     5|
|    305|    451|     3|
|      6|     86|     3|
|     62|    257|     2|
|    286|   1014|     5|
|    200|    222|     5|
|    210|     40|     3|
|    224|     29|     3|
|    303|    785|     3|
|    122|    387|     5|
|    194|    274|     2|
|    291|   1042|     4|
|    234|   1184|     2|
+-------+-------+------+
only showing top 20 rows



# Step 2: Data type conversions

In [0]:
dfs = df.withColumn("user_id", df["user_id"].cast(IntegerType()))
dfs.printSchema()

In [0]:
dfs1 = dfs.withColumn("item_id", df["item_id"].cast(IntegerType()))
dfs1.printSchema()

In [0]:
dfs2 = dfs1.withColumn("rating", df["rating"].cast(IntegerType()))
dfs2.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



# Step 3: Build Data Model

In [0]:
#Randomly splitting data into training and testing data
(training, test) = dfs2.randomSplit([0.8, 0.2])

In [0]:
# Build the recommendation model using ALS on the training data
# We set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(rank=10, #10 was by default
          maxIter=20, regParam=0.1,
          userCol="user_id", itemCol="item_id", ratingCol="rating",
          coldStartStrategy="drop",
          implicitPrefs=False)
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(training)
predictions1 = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error training = " + str(rmse))

rmse1 = evaluator.evaluate(predictions1)
print("Root-mean-square error test = " + str(rmse1))


Root-mean-square error training = 0.7381322297942123
Root-mean-square error test = 0.9207759730181222


# Step 4: Give recommendations based on the model

In [0]:
#Recommending top 10 movies for each user
%%time

top10movRecforusers = model.recommendForAllUsers(10)

CPU times: user 2.7 ms, sys: 0 ns, total: 2.7 ms
Wall time: 5.14 s


In [0]:
top10movRecforusersorted = top10movRecforusers.orderBy('user_id')
top10movRecforusersorted.show()

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|      1|[[1449, 5.1090508...|
|      2|[[1449, 4.869077]...|
|      3|[[320, 4.389251],...|
|      4|[[1642, 6.1412396...|
|      5|[[1268, 5.044817]...|
|      6|[[1643, 4.892668]...|
|      7|[[1643, 5.7340493...|
|      8|[[1449, 4.953917]...|
|      9|[[1367, 5.4910636...|
|     10|[[1643, 4.979647]...|
|     11|[[1344, 4.3760114...|
|     12|[[64, 5.124106], ...|
|     13|[[814, 4.896], [1...|
|     14|[[1344, 5.23139],...|
|     15|[[1450, 4.7111773...|
|     16|[[1643, 5.42787],...|
|     17|[[838, 4.335469],...|
|     18|[[1628, 4.7774544...|
|     19|[[1463, 5.5382385...|
|     20|[[1167, 4.8297143...|
+-------+--------------------+
only showing top 20 rows



# Step 5: Storing the output in a file

In [0]:
top10 = top10movRecforusersorted.withColumn("user_id", top10movRecforusersorted["user_id"].cast(StringType()))

In [0]:
top10mov = top10.withColumn("recommendations", top10["recommendations"].cast(StringType()))

In [0]:
top10mov.repartition(1).write.format("com.databricks.spark.csv").option("header", "true").save("/content/drive/My Drive/anedun_HW4")